In [4]:
import numpy as np
import pandas as pd 
import json
# import matplotlib as mpl 
# import matplotlib.cm as cm 
# import matplotlib.pyplot as plt 
# import plotly.graph_objects as go
# import seaborn as sns
import re 
np.random.seed(1)


In [5]:


# Json file lesen into list of dict
with open('data/Sarcasm_Headlines_Dataset.json', 'r') as sarcasm_headlines_file:
    dictionary_data = []
    
    # Read each line of the file
    for line in sarcasm_headlines_file:
        dictionary_data.append(json.loads(line))


#I will use this dictionary_data to create the np arrays and token dictionary
len(dictionary_data)

28619

In [6]:
# Using pondas to check for 
#(1) Non zero values in rows 
#(2) Are the labels only 0 and 1 
#(3) Make sure strings are not empty or only have a whitespace
#-------------------
#Class might be overkill. Oh well
class dataReader():
    def __init__(self,file_name):
        self.file = file_name
        self.data_frame = None
        
    #Parse the file into a dataframe
    #-------------------
    def create_df(self):
        json_objects = []
        with open(self.file, 'r') as file:
            for line in file:
                try:
                    json_object = json.loads(line)
                    json_objects.append(json_object)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON on line: {line.strip()}")
                    print(f"Error message: {str(e)}")
        self.data_frame = pd.DataFrame(json_objects)
        return self.data_frame

    # (1) Non zero values in rows
    #-------------------
    def remove_null_rows(self):
        size_before_removal = len(self.data_frame)
        self.data_frame = self.data_frame.dropna() 
        return f"Amount of rows of data_frame before removal of null rows: {size_before_removal} VS after removal : {len(self.data_frame)}"

    # (2) Check for the set of labels
    #-------------------
    def set_of_labels(self,column_name):
        label = self.data_frame[column_name]
        set_of_labels = label.unique()
        return f"Only the following labels exist : {set_of_labels}"

    # (3) No empty strings AND no whitespaces
    # My reasoning is that dropna() is checking for null which is  != ' ' and ''
    #-------------------
    def check_for_empty_strings(self,column_name):
        bool_non_empty_strings = ((df[column_name] != '').all and (df[column_name] != ' ').all())
        if bool_non_empty_strings == True:
            return f"No empty Strings in this column, nor are there any whitespaces"
        else:
            return f"yep Empty strings, better implement a solution"

In [7]:
obj = dataReader('data/Sarcasm_Headlines_Dataset.json')
df = obj.create_df()
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [8]:
obj.remove_null_rows()

'Amount of rows of data_frame before removal of null rows: 28619 VS after removal : 28619'

In [9]:
obj.set_of_labels("is_sarcastic")

'Only the following labels exist : [1 0]'

In [10]:
obj.check_for_empty_strings("headline")

'No empty Strings in this column, nor are there any whitespaces'

In [11]:
# The amount of rows hasn't changed. Thus, I will be using the list created from before

In [12]:
#Tokenization of words
dict_tokens = {}
train_size = int(0.2 * len(dictionary_data))
def getXY(data,size,dict_tokens):
    y_train = np.array([dictionary["is_sarcastic"] for dictionary in dictionary_data[:train_size] if dictionary["is_sarcastic"] in (0, 1)])


    max_Sequence_Length =0 
    word_pattern = r'\b\w+\b'  # Match whole words only
    punctuation_pattern = r'[^\w\s]'
    contraction_pattern = r"\b(?:[a-zA-Z]+(?:'[a-zA-Z]+)?)-?(?:[a-zA-Z]+(?:'[a-zA-Z]+)?)?\b"
    x_as_list = []
    combined_pattern = f'{contraction_pattern}|{word_pattern}|{punctuation_pattern}'
    for dictionary in dictionary_data[:train_size]:
        x = re.findall(combined_pattern, dictionary['headline'])
        if max_Sequence_Length < len(x):
            max_Sequence_Length = len(x)
        sequence_as_numbers = []
        for token in x:
            lowercase_token = token.lower()
            if token.lower() not in dict_tokens.keys():
                idx = (len(dict_tokens)) + 1
                dict_tokens[lowercase_token] = (len(dict_tokens)) + 1
                sequence_as_numbers.append(idx)
            else:
                sequence_as_numbers.append(dict_tokens[token])
        x_as_list.append(sequence_as_numbers)
    amountDataPoints = len(x_as_list)
    return x_as_list,y_train, amountDataPoints, max_Sequence_Length, dict_tokens
    
x_as_list , y, amountDataPoints, max_Sequence_Length,tokens= getXY(dictionary_data,train_size,dict_tokens)

#Padding to array
def add_Padding(data,amountDP,max_Sequence_Length):
    zeros_shape = (amountDP,max_Sequence_Length)
    np_input = np.zeros(zeros_shape)
    for idx , value in enumerate(x_as_list):  
        length_sequence = len(value)   
        sequence = np.array(value)
        
        #start_index = (max_Sequence_Length - length_sequence) 
        # I keep padding at the end for now 
        #---------------------------------------
        np_input[idx, :length_sequence] = sequence
    return np_input
X_padding = add_Padding(x_as_list,amountDataPoints,max_Sequence_Length)

In [13]:
# I forgot when i used this. I am just keeping it for now 
#---------------------------------------
# def reshaper(x):
#     # Get the dimensions of the input array
#     batch_size, seq_length = x.shape
    
#     d_model = 512
    
#     # Initialize a 3D array with zeros
#     batch = np.zeros((batch_size, 1, seq_length))
    
#     # Iterate over each row in the input array
#     for idx, row in enumerate(x):
#         print(row.tolist())

#         break
#         # Copy the row into the corresponding slice of the 3D array
#         batch[idx, 0, :] = row
        
#     return batch

In [14]:
#similar categories for words get similar 2D space with the embedding Matrix
#Vector for each existing token. randomized initial values
class begin_encoder():
    def __init__(self,dict_tokens, X_padding,max_sequence,d_model):

        self.max_sequence_length = max_sequence
        self.x_padding = X_padding
        self.dict = dict_tokens
        self.output = None
        self.embedding_matrix = None
        self.positional_encoding = None
        self.d_model = d_model
        self.batch_size = self.x_padding.shape[0]
        self.batch = None
        self.mask = None
        
    #Each dictionary token gets an embedding vector
    def create_each_embedding_vector(self):
        amountOfTokens = len(self.dict)
        self.embedding_matrix = np.random.randn(amountOfTokens,self.d_model)
        return self.embedding_matrix

    # X_padding reshape (DataPoints, Max sentence length) -> (Batch / Data, Words, embedded vector for word)
    def embed_sentences(self):  # Renamed the method to avoid conflict
        self.batch= np.zeros((self.batch_size, self.max_sequence_length, self.d_model))  # Changed 512 to self.d_model
        for idx, row in enumerate(self.x_padding):
            for pos, word_idx in enumerate(row):
                inedces = int(word_idx)
                if inedces == 0:
                    self.batch[idx,pos,:] = 0
                    break
                else:
                    self.batch[idx, pos, :] = self.embedding_matrix[inedces-1,:]
        return self.batch

    def mask_for_attention(self):
        self.mask = self.batch != 0
        return self.mask
    #I am not sure if the padded area wont change the outcome? since the sentences are at the end of padding and the sin and cos doesnt take that int account
    
    def positional_Encoding(self):
        col_em = self.embedding_matrix.shape[1]
        self.positional_encoding = np.zeros((self.max_sequence_length,col_em))
        position = np.arange(self.max_sequence_length).astype(np.float32).reshape(-1, 1)
        divisor = np.exp(np.arange(0, self.d_model, 2).astype(np.float32) * (-np.log(10000.0) / self.d_model))
        self.positional_encoding[:,0::2] = np.sin(position* divisor)
        self.positional_encoding[:, 1::2] = np.cos(position* divisor)
        return self.positional_encoding
        
    def encoding(self):
        self.output = np.zeros_like(self.batch, dtype=float) 
        for idx,dataPoint in enumerate(self.batch):
            self.output[idx] = dataPoint + self.positional_encoding
        return self.output


In [15]:
#after pos encoding and after multihead attention
class Layer_normalization():
    def __init__(self):
        self.eps = 10**-6
        self.weight = np.ones(1)
        self.bias = np.zeros(1)
    def forward(self,sequence):
        mean = np.mean(sequence,axis=1, keepdims=True)
        std = np.std(sequence,axis=1,keepdims=True)
        return (self.weight * (sequence - mean) / np.sqrt( (std**2+ self.eps))) + self.bias

In [16]:
#dropout i guess was so it randomly set avlues to 0 and reduces overfitting
#after normalization
class FeedForwardBlock:
    def __init__(self, d_model, d_ff, dropout):
        self.W1 = np.random.randn(d_model, d_ff)  # Weight matrix for the first linear transformation
        self.b1 = np.zeros(d_ff)  # Bias vector for the first linear transformation
        
        self.W2 = np.random.randn(d_ff, d_model)  # Weight matrix for the second linear transformation
        self.b2 = np.zeros(d_model)  # Bias vector for the second linear transformation
        self.dropout = dropout
        self.h1 = None
        self.h1_after_relu = None
        self.h2 = None
    def forward(self, x):
        # (batch_size, seq_len, d_model) --> (batch_size, seq_len, d_ff)
        h1 = np.matmul(x, self.W1) + self.b1
        self.h1 = h1
        h1_relu = np.maximum(h1, 0)  # ReLU activation for non linear function
        self.h1_after_relu = h1_relu
        # Apply dropout
        if self.dropout > 0:
            mask = np.random.rand(*h1.shape) < self.dropout
            h1 = np.where(mask, 0, h1)
    
        # (batch_size, seq_len, d_ff) --> (batch_size, seq_len, d_model)
        h2 = np.matmul(h1, self.W2) + self.b2
        self.h2 = h2
        return h2

    def backward(self, grad_loss, learning_rate):
        # Backpropagation
        pass

        return grad_h1


In [17]:
#mask e softmax to hide negative values will go close to 0
class MultiHeadAttentionBlock():
    def __init__(self,d_model,h,dropout):
        self.d_model = d_model
        self.h = h
        self.dropout = dropout
        assert d_model % h == 0 #make sure i can actually split the model into amount heads
        self.d_k = d_model // h
        self.w_q = np.random.randn(d_model, d_model)
        self.w_k = np.random.randn(d_model, d_model)
        self.w_v = np.random.randn(d_model, d_model)

        self.w_o = np.random.randn(d_model, d_model)
        self.dropout = dropout
        self.MH_a = None
        self.weighted_sum = None
        
    @staticmethod
    def attention(query, key, value, dropout):
        #d_k = len of col for each head
        d_k = query.shape[-1]
        
        # Calculate attention scores
        attention_scores = np.matmul(query, key.transpose((0, 1, 3, 2))) / np.sqrt(query.shape[-1])
    
        # Apply softmax
        attention_scores = np.exp(attention_scores - np.max(attention_scores, axis=-1, keepdims=True))
        attention_scores /= np.sum(attention_scores, axis=-1, keepdims=True)
       
        # Apply dropout
        if dropout > 0:
            mask = np.random.rand(*attention_scores.shape) < dropout
            attention_scores_applied_dropout = np.where(mask, 0, attention_scores)
    
        # Calculate weighted sum of values
        weighted_sum = np.matmul(attention_scores_applied_dropout, value)
    
        # Return weighted sum and attention scores
        attention_scores_applied_dropout = attention_scores_applied_dropout
        return weighted_sum, attention_scores_applied_dropout
        
    def forward(self, q, k, v, mask):
        q_snake = np.matmul(q , self.w_q)  # Apply linear transformation
        k_snake = np.matmul(k, self.w_k)  # Apply linear transformation
        v_snake = np.matmul(v, self.w_v)  # Apply linear transformation
        batch_size, seq_len, _ = q_snake.shape

        # Reshape query, key, and value for multi-head attention
        if mask is not None:
            masked_q = np.where(mask == False, -1e9, q_snake)
            masked_k= np.where(mask == False, -1e9, k_snake)
            masked_v = np.where(mask == False, -1e9,v_snake)
        #  mask_reshaped = np.reshape(mask, (batch_size, seq_len, self.h, self.d_k))

        q_snake = np.reshape(q_snake, (batch_size,seq_len, self.h, self.d_k))
        k_snake= np.reshape(k_snake, (batch_size, seq_len, self.h, self.d_k))
        v_snake = np.reshape(v_snake, (batch_size,seq_len, self.h, self.d_k))
        # Transpose query, key, and value for multi-head attention
        # mask_transposed = np.transpose(mask_reshaped,(0,2,1,3))       
        query_transposed = np.transpose(q_snake, (0, 2, 1, 3))  # (batch_size, h, seq_len, d_k)
        key_transposed = np.transpose(k_snake, (0,2, 1, 3))  # (batch_size, h, seq_len, d_k)
        value_transposed = np.transpose(v_snake, (0, 2, 1, 3))  # (batch_size, h, seq_len, d_k)
        
        # Calculate attention
        x, attention_scores = MultiHeadAttentionBlock.attention(query_transposed, key_transposed, value_transposed, self.dropout)
        # Combine all the heads together
        x = np.transpose(x, (0, 2, 1, 3))  # (batch_size, seq_len, h, d_k)
        x = np.reshape(x, (batch_size, seq_len, -1))  # (batch_size, seq_len, h * d_k)` 
        self.MH_a = np.matmul(x,self.w_o)
        return self.MH_a, attention_scores
    
    def backward(self, grad_output, learning_rate):
        pass


        return grad_input_q, grad_input_k, grad_input_v


In [18]:
#connection from norm and mutlihead to Feedforward and to -> norm that is after feedforward 
class ResidualConnection:
    def __init__(self, dropout):
        self.dropout= dropout
        self.norm = Layer_normalization()  # Assuming LayerNormalization is defined elsewhere

    def forward(self, x, sublayer):
        # Apply dropout manually
        if self.dropout > 0:
            mask = np.random.rand(*x.shape) < self.dropout
            x_drop = np.where(mask, 0, x)
        else:
            x_drop = x
        
        # Apply layer normalization
        x_norm = self.norm.forward(x_drop)
        
        # Apply sublayer transformation
        x_sub = sublayer(x_norm)

        # Add residual connection
        return x + x_sub
    


In [19]:
class EncoderBlock():
    def __init__(self, dict_tokens, max_sequence, d_models , num_heads, d_ff, dropout):
        self.x_padded = None
        self.dict_tokens = dict_tokens
        self.max_sequence = max_sequence
        self.d_model = d_models
        self.self_attention = None
        self.feed_forward = None
        self.residual_connection_after_mha = None
        self.residual_connection_after_ff= None
        self.heads = num_heads
        self.dropout = dropout
        self.d_ff = d_ff
        self.output = None
        self.embedding_matrix_grads = None
        self.embedding_matrix = None
        self.attention_scores = None
        
    def forward(self, X_padding):
        self.x_padded = X_padding
       
        #===================================
        #begin_encoding_ positional + embedding
        #Here i get the embedding vector for the words
        encoder = begin_encoder(self.dict_tokens,self.x_padded,self.max_sequence,self.d_model)
        encoder.create_each_embedding_vector()
        x_embeddings = encoder.embed_sentences()
        self.embedding_matrix = x_embeddings
        # Shape : (32, 51, 512) (batch, sentence length, embedding vector)
        #===================================
        
        mask = encoder.mask_for_attention()
        encoder.positional_Encoding()
        positional_encoded_input = encoder.encoding()
        axes = self.x_padded.shape[0]
        #On all the 32 datapoints
        mask_batched = mask[0:axes,:,:]
        print(mask_batched.shape)
        
        #===================================
        #MHA
        self.self_attention = MultiHeadAttentionBlock(self.d_model,self.heads,self.dropout)
        mha, self.attention_scores = self.self_attention.forward(x_embeddings, x_embeddings, x_embeddings, mask_batched)
        self.residual_connection_after_mha = ResidualConnection(self.dropout)
        multi_head_normalized = self.residual_connection_after_mha.forward(x_embeddings, lambda x: self.self_attention.forward(x_embeddings, x_embeddings, x_embeddings, mask_batched)[0])
        

        #======================================
        #feedforward
        self.feed_forward = FeedForwardBlock(self.d_model,self.d_ff,self.dropout)
        self.residual_connection_after_ff= ResidualConnection(self.dropout)
        self.output = self.residual_connection_after_ff.forward(multi_head_normalized, lambda x:self.feed_forward.forward(multi_head_normalized))
        return self.output
    
    def backward(self, loss_from_classifier,learning_rate):
        pass

        print(q,k,v)
    def update_embedding_vectors(self, learning_rate):
        # Update embedding vectors using gradient descent
        self.embedding_matrix -= learning_rate * self.embedding_matrix_grads
        

In [119]:
# class SoftmaxActivation:
#     def __init__ (self):
#         self.output = None
#     def forward(self,input):
#         #exp is super high we subtract the max value so its only 0 or negative values which on the exp curve can only go up to 1
#         exp = np.exp(input - np.max(input,axis =1 , keepdims = True) )
#         sum = np.sum(exp,axis =1 , keepdims =True)
#         self.output = exp / sum

#         return self.output
#     def backward(self, grad):
#         J = np.diag(self.output.flatten()) - np.dot(self.output.T, self.output)
#         # Compute gradient of the loss with respect to the input of softmax
#         grad_input = np.dot(grad, J)
#         return grad_input


class Layerdense:
    def __init__(self,n_inputs,n_nodes,amount_data):
        self.amount_data = amount_data
        self.n_inputs = n_inputs
        self.n_neurons = n_nodes
        self.weights = 0.1 * np.random.randn(n_inputs,self.n_neurons)
        self.bias = np.random.randn(1,self.n_neurons)
        self.inputs = None

        self.derivative_inputs = None
        self.derivative_bias = None
        self.derivative_weights = None
        
    def forward(self,inputs):
        #input shape = batchsize x sequence max length x embedding vector length d_model
        self.inputs = inputs
        self.output = np.dot(self.inputs,self.weights) + self.bias
        return self.output

    def backward(self, grad_output, learning_rate):
        # self.inputs.shape (32,5) 
        # self.weights (5, 2) weights
        # derivative respect to w = X * derivative of Loss
        # grad_ouput shape = (32,2)
        #==================================================
        self.derivative_weights = np.dot(self.inputs.T,grad_output)
        
        # Why did chatgpt tell me derivative of bias is usually sum over grad_output?)
        # derivative bias = w x + b => 1 * loss of the successor layer/loss 
        #=========================
        self.derivative_bias =  grad_output.mean(axis=0)
        # Shape weights = (5, 2) 
        self.derivative_inputs = np.dot(grad_output,self.weights.T)

        # UPDATING the weights and biases
        # self.weights (5, 2) weights
        #===============================
        self.weights = self.weights - learning_rate * self.derivative_weights
        self.bias = self.bias - learning_rate * self.derivative_bias
        
        # grad_input = np.dot(grad_output, self.weights.T)
        # grad_weights = np.dot(self.inputs.T, grad_output)
        # self.weights -= learning_rate * grad_weights
        # self.bias -= learning_rate * grad_output.mean(axis=0)
        return self.derivative_inputs


class ActivationSigmoid:
    def __init__(self):
        self.output = None

    def forward(self, x):
        # Apply the log-sum-exp trick
        maximum = x.max()
        x -= maximum
        self.output = 1 / (1 + np.exp((x)))
        return self.output
    def backward(self,x , grad):
        # sigmoid * (1- sigmoid)
        derivative = grad * 1 / (1 + np.exp((x))) * ( 1 - (1 / (1 + np.exp((x)))))
        return derivative

class Classification:
    def __init__(self, input_size, hidden_size, output_size, encoded_inputs):
        self.batch_size = encoded_inputs.shape[0]
        self.sequence_length = encoded_inputs.shape[1]
        self.input = encoded_inputs
        
        self.layer1 = Layerdense(input_size, hidden_size, self.batch_size)
        self.activation1 = ActivationSigmoid()
        self.layer2 = Layerdense(hidden_size, output_size, self.batch_size)
        self.activation2 = ActivationSigmoid()

    def compute_loss(self,target):
        predictions = self.activation2.output
        cross_entropy_loss = -np.mean(target * np.log(predictions + 1e-9) + (1 - target) * np.log(1 - predictions + 1e-9))
        return cross_entropy_loss
        
    def forward(self):
        self.layer1.forward(self.input)
        self.activation1.forward(self.layer1.output)
        self.layer2.forward(self.activation1.output)
        #binary classification use sigmoid
        self.activation2.forward(self.layer2.output)
        return self.activation2.output

    def backward(self, target, learning_rate):
        # Small epsilon value to avoid division by zero
        epsilon = 1e-7  
        assert self.activation2.output.shape[0] != 0, f'cant div by {self.activation2.output.shape[0]}'
        derivative_cross_entropy = ((self.activation2.output - target) / (self.activation2.output + epsilon)  * ( 1- self.activation2.output + epsilon)) / self.activation2.output.shape[0]
        #derivative Cross entropy shape (32,2) 
        gradients_layer1 = self.layer2.backward(derivative_cross_entropy, learning_rate)
        
        gradients_activation1 = self.activation1.backward(self.layer1.output,gradients_layer1)
        grad_layer = self.layer1.backward(gradients_activation1, learning_rate)
        return grad_layer


In [120]:
def compute_encoder_loss(encoder_output, y, predictions):
    # Predict using the classifier

    cross_entropy_loss = -np.mean(y * np.log(predictions + 1e-9) + (1 - y) * np.log(1 - predictions + 1e-9))

    return cross_entropy_loss

In [121]:
# Set hyperparameters
batch_size = 32  
learning_rate_encoder = 0.0001  
learning_rate_classifier = 0.001  
num_epochs = 10  
d_model = 512
num_heads = 4
d_ff = 1000
dropout = 0.1
# Instantiate EncoderBlock
encoder = EncoderBlock(tokens, max_Sequence_Length, d_model, num_heads, d_ff, dropout)

accuracy1_list = []
accuracy2_list = []
# Loop over epochs
for epoch in range(num_epochs):
    # Shuffle the data for each epoch
    indices = np.arange(len(X_padding))
    np.random.shuffle(indices)
    X_train_shuffled = X_padding[indices]
    y_train_shuffled = y[indices]
    
    # Loop over mini-batches
    for i in range(0, 64, batch_size):
        # Extract mini-batch
        X_batch = X_train_shuffled[i:i+batch_size]
        y_batch = y_train_shuffled[i:i+batch_size]

        # Forward pass for encoder BLock transformer
        encoder_output = encoder.forward(X_batch)
        encoder_output_pooled = np.mean(encoder_output,axis=1)
        print(encoder_output_pooled.shape ,'encoded')
        classifier = Classification(d_model, 5, 2, encoder_output_pooled)
        
        pred = classifier.forward()
        pred[pred >0.6] = 1
        pred[pred <0.6] = 0

        y_batch = y_batch.reshape((-1,1))
        
        loss = classifier.compute_loss(y_batch)
        
        gradient = classifier.backward(y_batch,learning_rate_classifier)

        # Compute loss for encoder
        grad_encoder = encoder.backward(gradient,learning_rate_classifier)
    
        # Forward pass for classifier


        #2nd neuron
        accuracy2 = np.mean(pred[:,1] == y_batch)
        accuracy1 = np.mean(pred[:,0] == y_batch)
        accuracy1_list.append(accuracy1)
        accuracy2_list.append(accuracy2)
plt.plot(range(num_epochs), accuracy1_list[:num_epochs], label='Neuron 1 Accuracy')
plt.plot(range(num_epochs), accuracy2_list[:num_epochs], label='Neuron 2 Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Over Epochs')
plt.legend()
plt.show()

(32, 51, 512)
(32, 512) encoded
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
10.361632917973203 here ()
(32, 512) gradloss Shapem (32, 51, 1000)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 51 is different from 512)